In [136]:
import pandas as pd 
import geopandas as gpd 
from shapely.geometry import Point
import numpy as np 

In [137]:
# Load Wilayas Features and calculate polygon centroids
wilayas_gjson = '../dz-admin/wilayas_48.geojson'
wilayas_gdf = gpd.read_file(wilayas_gjson)
wilayas_gdf = wilayas_gdf[['code', 'nom_maj', 'geometry']]
wilayas_gdf['nom'] = wilayas_gdf.nom_maj.str.lower()
wilayas_gdf = wilayas_gdf.rename(columns={'nom': 'nom_wil', 'code': 'code_wil'})
wilayas_gdf = wilayas_gdf.drop(columns=['nom_maj'], axis=1)
wilayas_gdf['centroid'] = wilayas_gdf['geometry'].centroid
wilayas_gdf.head()

<ipython-input-137-9dd2fb12b137>:8: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  wilayas_gdf['centroid'] = wilayas_gdf['geometry'].centroid


,code_wil,geometry,nom_wil,centroid
0,1,"MULTIPOLYGON (((1.33461 20.72282, 1.32667 20.7...",adrar,POINT (-0.62174 26.01400)
1,2,"MULTIPOLYGON (((1.55326 36.02990, 1.55325 36.0...",chlef,POINT (1.25669 36.23826)
2,3,"MULTIPOLYGON (((2.88464 32.88137, 2.87848 32.8...",laghouat,POINT (2.72955 33.60757)
3,4,"MULTIPOLYGON (((7.47320 35.52005, 7.47227 35.5...",oum el bouaghi,POINT (7.06527 35.83646)
4,5,"MULTIPOLYGON (((6.10102 35.05459, 6.10090 35.0...",batna,POINT (5.87455 35.41969)


In [138]:
wilayas_gdf = wilayas_gdf.set_crs("EPSG:4326")
wilayas_gdf.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [139]:
# Load Tagreted dataset and set to_check to False assuming all geocoded coordinates are OK 
pois_path = '../dz-datasets/accountants/'
pois = pd.read_csv(pois_path + 'personnes_physiques.csv')
pois['wilaya_low'] = pois.wilaya.str.lower()
pois['to_check'] = False
pois.head()

,num,num_inscription,nom,prenom,adresse,wilaya,tel,fax,email,lat,lon,wilaya_low,to_check
0,301,1387,BELHADEF,IMAD,CITE LA PAIE - ILLIZI,Illizi,29421397,29421397,NaN,26.477586,8.477633,illizi,False
1,1383,1475,KHENGAOUI,SMAÎL,HAI EL WASSATE BP 129 - ILLIZI,Illizi,29421691,NaN,khengaoui@gmail.com,26.477586,8.477633,illizi,False
2,1178,1563,HAMDI,BACHIR,RUE EMIR ABDELKADER - ILLIZI,Illizi,29421404,29760356,NaN,26.500030,8.471065,illizi,False
3,2080,1835,ZEGGAI,RACHED,"RUE EMIR ABDELKADER, CENTRE VILLE- ILLIZI",Illizi,NaN,NaN,NaN,26.500030,8.471065,illizi,False
4,739,1107,BOUZELKOUT,ABDELMADJID,"PLACE DE ANNABA, EL KALA - EL TAREF",El Tarf,38660510,38660510,I,36.895509,8.452496,el tarf,False


In [140]:
# Convert pandas DataFrame to GeoPandas 
pois['geometry'] = pois.apply(lambda row: Point(row.lon, row.lat), axis=1)
pois_gdf  = gpd.GeoDataFrame(pois)
pois_gdf.head()

,num,num_inscription,nom,prenom,adresse,wilaya,tel,fax,email,lat,lon,wilaya_low,to_check,geometry
0,301,1387,BELHADEF,IMAD,CITE LA PAIE - ILLIZI,Illizi,29421397,29421397,NaN,26.477586,8.477633,illizi,False,POINT (8.47763 26.47759)
1,1383,1475,KHENGAOUI,SMAÎL,HAI EL WASSATE BP 129 - ILLIZI,Illizi,29421691,NaN,khengaoui@gmail.com,26.477586,8.477633,illizi,False,POINT (8.47763 26.47759)
2,1178,1563,HAMDI,BACHIR,RUE EMIR ABDELKADER - ILLIZI,Illizi,29421404,29760356,NaN,26.500030,8.471065,illizi,False,POINT (8.47106 26.50003)
3,2080,1835,ZEGGAI,RACHED,"RUE EMIR ABDELKADER, CENTRE VILLE- ILLIZI",Illizi,NaN,NaN,NaN,26.500030,8.471065,illizi,False,POINT (8.47106 26.50003)
4,739,1107,BOUZELKOUT,ABDELMADJID,"PLACE DE ANNABA, EL KALA - EL TAREF",El Tarf,38660510,38660510,I,36.895509,8.452496,el tarf,False,POINT (8.45250 36.89551)


In [141]:
pois_gdf.count()

num                2116
num_inscription    2116
nom                2116
prenom             2116
adresse            2116
wilaya             2116
tel                1389
fax                 994
email              1477
lat                2027
lon                2027
wilaya_low         2116
to_check           2116
geometry           2116
dtype: int64

In [142]:
# Set the Coordinate reference system
pois_gdf = pois_gdf.set_crs("EPSG:4326")
pois_gdf.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [143]:
# Check for rows with no coordiantes
no_coords_gdf = pois_gdf[(pois_gdf['lat'].isnull() | pois_gdf['lat'].isnull())]

# Merge with Wialyas DataFrame and set geometry to centroids
# The merge may be done either with Wilayas code or wilayas names   
no_coords_gdf = no_coords_gdf.merge(wilayas_gdf, left_on='wilaya_low', right_on='nom_wil', how='left')
no_coords_gdf['geometry'] = no_coords_gdf['centroid']
no_coords_gdf['to_check'] = True

with_coords_gdf = no_coords_gdf.drop(columns=['geometry_x'], axis=1)

with_coords_gdf.count() 

num                89
num_inscription    89
nom                89
prenom             89
adresse            89
wilaya             89
tel                61
fax                42
email              59
lat                 0
lon                 0
wilaya_low         89
to_check           89
code_wil           89
geometry_y         89
nom_wil            89
centroid           89
geometry           89
dtype: int64

In [144]:
# Update the Targeted DataFrame by adding Wilayas geometries and centroids 
pois_gdf = pois_gdf[(pois_gdf['lat'].notnull() & pois_gdf['lat'].notnull())]

# Merge with Wialyas DataFrame and set geometry to centroids
# The merge may be done either with Wilayas code or wilayas names   
pois_gdf = pois_gdf.merge(wilayas_gdf, left_on='wilaya_low', right_on='nom_wil', how='left')
pois_gdf.rename(columns={"geometry_x": "geometry"}, inplace=True)
pois_gdf.count()

num                2027
num_inscription    2027
nom                2027
prenom             2027
adresse            2027
wilaya             2027
tel                1328
fax                 952
email              1418
lat                2027
lon                2027
wilaya_low         2027
to_check           2027
geometry           2027
code_wil           2027
geometry_y         2027
nom_wil            2027
centroid           2027
dtype: int64

In [145]:
# Concat the Targeted DataFrame with rows without coordinates and approximated with Wilayas centroids 
pois_gdf = pd.concat([pois_gdf, with_coords_gdf])

# Drop unecessary columns
pois_gdf = pois_gdf.drop(columns=['centroid', 'nom_wil'], axis=1)

pois_gdf[(pois_gdf['lat'].isnull() | pois_gdf['lon'].isnull())]
pois_gdf.count()

num                2116
num_inscription    2116
nom                2116
prenom             2116
adresse            2116
wilaya             2116
tel                1389
fax                 994
email              1477
lat                2027
lon                2027
wilaya_low         2116
to_check           2116
geometry           2116
code_wil           2116
geometry_y         2116
dtype: int64

In [146]:
# Sort the DataFrame by code (Wilaya number) and reset the index
pois_gdf = pois_gdf.sort_values(by=['code_wil'],ignore_index=True)
pois_gdf.reset_index(drop=True)
pois_gdf.head()

,num,num_inscription,nom,prenom,adresse,wilaya,tel,fax,email,lat,lon,wilaya_low,to_check,geometry,code_wil,geometry_y
0,1865,2038,SAIDI,ABDERRAFIR,RUE ABDELKA DER BENSLIMANE EN FACE PARC DTP - ...,Adrar,NaN,NaN,saidiabderaflr@gmail.com,33.618970,-7.130554,adrar,False,POINT (-7.13055 33.61897),1,"MULTIPOLYGON (((1.33461 20.72282, 1.32667 20.7..."
1,239,1511,BAKHOUIA,ABDERRAHMANE,"CITE 137 LOGTS, 1er MAI - ADRAR",Adrar,49966472,49966472,ab647200@yahoo.fr,28.017440,-0.264250,adrar,False,POINT (-0.26425 28.01744),1,"MULTIPOLYGON (((1.33461 20.72282, 1.32667 20.7..."
2,1418,1355,LAAFOU,AHMED,CITE 175 LOGTS ROUTETILILANE - ADRA R,Adrar,0499667OO,NaN,NaN,27.898056,-0.248333,adrar,False,POINT (-0.24833 27.89806),1,"MULTIPOLYGON (((1.33461 20.72282, 1.32667 20.7..."
3,61,1632,AHRES,HASSEN,RUE KADDOUR BELLITIM - ADRAR,Adrar,49964925,NaN,hassanahres@gmail.com,27.882773,-0.270904,adrar,False,POINT (-0.27090 27.88277),1,"MULTIPOLYGON (((1.33461 20.72282, 1.32667 20.7..."
4,327,1213,BELLAOUI,ABDERRAHMANE,CITE 05 JUILLET 1962 - ADRAR,Adrar,NaN,49951555,NaN,28.017440,-0.264250,adrar,False,POINT (-0.26425 28.01744),1,"MULTIPOLYGON (((1.33461 20.72282, 1.32667 20.7..."


In [147]:
# Check if coordinates are within Wilayas polygons
pois_gdf['coords_in_wil'] = [pois_gdf.geometry[i].within(pois_gdf.geometry_y[i]) for i in range(pois_gdf.geometry.count())] 
pois_gdf[pois_gdf.coords_in_wil == False]

,num,num_inscription,nom,prenom,adresse,wilaya,tel,fax,email,lat,lon,wilaya_low,to_check,geometry,code_wil,geometry_y,coords_in_wil
0,1865,2038,SAIDI,ABDERRAFIR,RUE ABDELKA DER BENSLIMANE EN FACE PARC DTP - ...,Adrar,NaN,NaN,saidiabderaflr@gmail.com,33.618970,-7.130554,adrar,False,POINT (-7.13055 33.61897),1,"MULTIPOLYGON (((1.33461 20.72282, 1.32667 20.7...",False
6,1086,801,GRISI,MOHAMED,CIA DES FONCTIONNAIRES BT 05 N'C 06 - CHLEF,Chlef,27777822,NaN,NaN,36.417984,-79.650211,chlef,False,POINT (-79.65021 36.41798),2,"MULTIPOLYGON (((1.55326 36.02990, 1.55325 36.0...",False
55,1234,668,HARIZI,SMAIL,CITE 24 LOGTS SONATIBA EN FACE DE 'OPGI -OUM...,Oum El Bouaghi,32426983,NaN,harizlsmail@yahoo.fr,36.298662,6.620393,oum el bouaghi,False,POINT (6.62039 36.29866),4,"MULTIPOLYGON (((7.47320 35.52005, 7.47227 35.5...",False
64,487,1277,BENSIDI BRAHIM,DJAMEL,SITE 1N°276 PRES DE LA MOSQUEE KHADIDJA OEM BP...,Oum El Bouaghi,32492869,32492869,bensidibrahlm@hotmall.fr,38.917136,-77.056712,oum el bouaghi,False,POINT (-77.05671 38.91714),4,"MULTIPOLYGON (((7.47320 35.52005, 7.47227 35.5...",False
69,1088,515,GUECHI,RACHID,"03, RUE BENAGGOUNE MAAMER, CITE ANNASSER - BATNA",Batna,33867917,33867917,guechirad@yahoo.fr,36.864597,10.156320,batna,False,POINT (10.15632 36.86460),5,"MULTIPOLYGON (((6.10102 35.05459, 6.10090 35.0...",False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2003,260,2097,BEDDIAR,BADRA,"LOTS ROUTE DE ANNABA N'Ol,TRANCHE N' 67 - SO...",Souk Ahras,37820731,NaN,badrabeddiar@gmail.com,36.914208,7.742667,souk ahras,False,POINT (7.74267 36.91421),41,"MULTIPOLYGON (((7.53798 35.88014, 7.53725 35.8...",False
2006,1572,2003,MEHNAOUI,NOUREDDINE,CITE 26 AVRIL· 12 LOCAUX COMMERCIAUX - 50UK-AHRAS,Souk Ahras,37752182,37752509,NaN,45.522642,-73.520754,souk ahras,False,POINT (-73.52075 45.52264),41,"MULTIPOLYGON (((7.53798 35.88014, 7.53725 35.8...",False
2076,594,1736,BOUDEBRA,BAKIR,"CITE BABA SAAD EL GHARBI,CIDEX N'OS BP 123 - G...",Ghardaia,NaN,NaN,CACBAKIR@GMAIL.COM,36.068909,14.201810,ghardaia,False,POINT (14.20181 36.06891),47,"MULTIPOLYGON (((4.12740 33.01380, 4.14386 33.0...",False
2080,578,933,BOUCHELAGHAM,NADIR,"RUE BESLIMANE BRAHIM, SERRIANE - GHARDAIA",Ghardaia,29890464,29890464,b_nadirl@yahoo.fr,33.618970,-7.130554,ghardaia,False,POINT (-7.13055 33.61897),47,"MULTIPOLYGON (((4.12740 33.01380, 4.14386 33.0...",False


In [148]:
# Update to_check flag to take into account points outside Wilays and without geocoding coordinates
final_gdf = pois_gdf
final_gdf['to_check'] = (pois_gdf['to_check'] == True) | (pois_gdf['coords_in_wil'] == False)   

# Remove unecessary columns
final_gdf = final_gdf.drop(['geometry_y', 'coords_in_wil', 'wilaya_low'], axis=1)

final_gdf[final_gdf.to_check == True]

,num,num_inscription,nom,prenom,adresse,wilaya,tel,fax,email,lat,lon,to_check,geometry,code_wil
0,1865,2038,SAIDI,ABDERRAFIR,RUE ABDELKA DER BENSLIMANE EN FACE PARC DTP - ...,Adrar,NaN,NaN,saidiabderaflr@gmail.com,33.618970,-7.130554,True,POINT (-7.13055 33.61897),1
6,1086,801,GRISI,MOHAMED,CIA DES FONCTIONNAIRES BT 05 N'C 06 - CHLEF,Chlef,27777822,NaN,NaN,36.417984,-79.650211,True,POINT (-79.65021 36.41798),2
55,1234,668,HARIZI,SMAIL,CITE 24 LOGTS SONATIBA EN FACE DE 'OPGI -OUM...,Oum El Bouaghi,32426983,NaN,harizlsmail@yahoo.fr,36.298662,6.620393,True,POINT (6.62039 36.29866),4
64,487,1277,BENSIDI BRAHIM,DJAMEL,SITE 1N°276 PRES DE LA MOSQUEE KHADIDJA OEM BP...,Oum El Bouaghi,32492869,32492869,bensidibrahlm@hotmall.fr,38.917136,-77.056712,True,POINT (-77.05671 38.91714),4
69,1088,515,GUECHI,RACHID,"03, RUE BENAGGOUNE MAAMER, CITE ANNASSER - BATNA",Batna,33867917,33867917,guechirad@yahoo.fr,36.864597,10.156320,True,POINT (10.15632 36.86460),5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2077,1413,147,KOUZRIT,BRAHIM,AVENUE DU 1er MAI - GUARDAIAI,Ghardaia,NaN,NaN,kouzrit50@gmail.com,NaN,NaN,True,POINT (3.17728 31.18000),47
2080,578,933,BOUCHELAGHAM,NADIR,"RUE BESLIMANE BRAHIM, SERRIANE - GHARDAIA",Ghardaia,29890464,29890464,b_nadirl@yahoo.fr,33.618970,-7.130554,True,POINT (-7.13055 33.61897),47
2100,430,1877,BENKHELIFA,ALI,"26, AVENUE DU 1er NOVEMBRE -GHARDAIA",Ghardaia,29234702,29234703,alibenkhelifa@yahoo.fr,NaN,NaN,True,POINT (3.17728 31.18000),47
2108,1130,1666,HADOAD,AHMED,RUE HAMANI BELOJILALI - GUELIZANE,Relizane,46718057,NaN,NaN,36.770662,3.056956,True,POINT (3.05696 36.77066),48


In [149]:
final_gdf.count()

num                2116
num_inscription    2116
nom                2116
prenom             2116
adresse            2116
wilaya             2116
tel                1389
fax                 994
email              1477
lat                2027
lon                2027
to_check           2116
geometry           2116
code_wil           2116
dtype: int64

In [150]:
# Write to a GeoJSON file

gpd.GeoDataFrame(final_gdf).to_file(pois_path + "pois_checked_in_out_wilayas.geojson", driver='GeoJSON')